# RANSAC-ing and reverse engineering point cloud data 
### Ann Arbor Scientific and Technical Computing
### August 27, 2020 

<img src="img/ransac_example.png" style="height:100%; width:100%"></img>

# What is RANSAC?
<img src="img/surface_normals.jpg" align="right" style="height:500px"></img>
<br clear="left">
RANSAC (or, Random Sample Consensus) is a way of fitting basic geometric shapes like planes, spheres, cones, and torii (primitive shapes) to point cloud data. Point cloud data is a series of (x,y,z) coordinates that describe some object or objects. For instance, the sphere on the right shows what we recognize as a sphere, but to a computer would be just a collection of coordinates with no higher meaning. RANSAC also relies on the surface normals, also seen in the graph pointing straight out of the sphere. The normals help the algorithm distinguish between primitives. For instance, by selecting three points you could define either a sphere or a plane, but the normals would determine which underlying shape is more likely.

The random part of the algorithm has to do with how RANSAC determines which primitives best model the point cloud. The process has to be randomized as trying every combination over ever subset of points would be prohibitively slow and computationally expensive. This is also why consensus is part of the name; the algorithm scores the candidate fits and tries to select those most agreed on by the random samples.


# Why do we need it?

<div class="row", style="display:flex">
    <div class="column" style="flex:33%">
        <img src="img/pmi_cad.gif" align="top" style="height:200px"></img>
        <br clear="top">
        <h3 style="text-align:center">Pulling out contextual information</h3>
    </div>
    <div class="column" style="width:33%">
        <img src="img/lidar-cloud.gif" align="top" style="height:200px"></img>
        <br clear="top">
        <h3 style="text-align:center">Aiding computer vision</h3>
    </div>
    <div class="column" style="width:33%">
        <img src="img/noisy_point_cloud.gif" align="top" style="height:200px"></img>
        <br clear="top">
        <h3 style="text-align:center">De-noising and refining computer graphics</h3>
    </div>
</div>    


# How does it work?

<img src="img/octree_rabbit.jpeg" align="right" hspace="10px" vspace="10px"></img>
<br clear="left">
<ul>
    <li>Build an octree partitioning over the space of the scanned data (see rabbit on right)</li>
    <li>From within those cells, do a random sampling</li>
    <li>Do the sampled points fit a defined surface type?</li>
    <li>Do their normals align as well?</li>
    <li>Score the fits for each primitive, as well as a measure of fit likelihood</li>
    <li>Move up the octree (include larger and larger cells) until a clear winner emerges</li>
    <li>Once a winning primitive is selected, find all points that would be included and refine the shape parameters</li>
    <li>Remove the points on this primitive from the search space and continue with other candidate primitives</li>
</ul>

With this high level understanding of how the algorithm works. Let's see it in practice using [RANSAC.jl and the exellent examples](https://csertegt3.github.io/RANSAC.jl/stable/example/) provided by the author.

In [1]:
using RANSAC
using FileIO 
using MeshIO

In [2]:
# Load points from file
m = load("data/fandisk_input.obj");
# Load points into RANSAC data type
pc = RANSACCloud(m.position, m.normals, 8);

<img src="img/surface_mesh.png" style="height:650px"></img>

In [3]:
# Generate default parameters
p = ransacparameters()

(iteration = (drawN = 3, minsubsetN = 15, prob_det = 0.9, shape_types = UnionAll[FittedPlane, FittedCone, FittedCylinder, FittedSphere], τ = 900, itermax = 1000, extract_s = :nofminset, terminate_s = :nofminset), common = (collin_threshold = 0.2, parallelthrdeg = 1.0), plane = (ϵ = 0.3, α = 0.08726646259971647), cone = (ϵ = 0.3, α = 0.08726646259971647, minconeopang = 0.03490658503988659), cylinder = (ϵ = 0.3, α = 0.08726646259971647), sphere = (ϵ = 0.3, α = 0.08726646259971647, sphere_par = 0.02))

In [4]:
# Tweak and update parameters
newparams = (ϵ=0.05, α=deg2rad(10),)

p = ransacparameters(p, sphere=newparams, cone=newparams, plane=newparams, cylinder=newparams, iteration=(τ=50, itermax=100_000,))

(iteration = (drawN = 3, minsubsetN = 15, prob_det = 0.9, shape_types = UnionAll[FittedPlane, FittedCone, FittedCylinder, FittedSphere], τ = 50, itermax = 100000, extract_s = :nofminset, terminate_s = :nofminset), common = (collin_threshold = 0.2, parallelthrdeg = 1.0), plane = (ϵ = 0.05, α = 0.17453292519943295), cone = (ϵ = 0.05, α = 0.17453292519943295, minconeopang = 0.03490658503988659), cylinder = (ϵ = 0.05, α = 0.17453292519943295), sphere = (ϵ = 0.05, α = 0.17453292519943295, sphere_par = 0.02))

In [5]:
# run the RANSAC algorithm
extr, _ = ransac(pc, p, true, reset_rand=true);

In [6]:
using RANSACVisualizer

In [7]:
display(showshapes(pc, extr; show_axis = false, markersize=75)); #opens external window

<img src="img/results_by_surface.png" style="width:650px"></img>

In [8]:
display(showbytype(pc, extr; show_axis = false, markersize=75)); #opens external window

<img src="img/results_by_shape.png" style="width:650px"></img>

### Sources
- [Efficient RANSAC for Point-Cloud Shape Detection](https://cg.cs.uni-bonn.de/aigaion2root/attachments/schnabel-2007-efficient.pdf)
- [RANSAC.jl](https://github.com/cserteGT3/RANSAC.jl)
- [RANSAC JuliaCon2020 presentation](https://www.youtube.com/watch?v=EcuqNhmfovM)